In [44]:
from ultralytics import YOLO
import cv2
import numpy as np
model = YOLO('camoflage.pt')
img_path = "C:\\Users\\adhik\\Downloads\\camouflage\\dataset-splitM\\data\\image948.jpg"
img = cv2.imread(img_path)
if img is None:
    raise ValueError("Image not found. Check the file path.")
target_size = (640, 640)
img_resized = cv2.resize(img, target_size)
results = model.predict(source=img_resized, conf=0.10)
output_mask = np.zeros_like(img_resized)
total_iou = 0
total_precision = 0
total_recall = 0
count = 0
for result in results:
    if result.masks is not None:
        for i, mask in enumerate(result.masks):
            mask_np = mask.data.numpy() 
            
            if mask_np.size == 0:
                continue 

            
            if mask_np.ndim == 3:
                mask_np = mask_np[0] 
            
           
            mask_binary = mask_np > 0 
            output_mask[mask_binary] = (0,255,255) 
            gt_mask = np.zeros_like(mask_binary)  
            gt_mask[100:300, 100:300] = 1  
            intersection = np.logical_and(mask_binary, gt_mask).sum()
            union = np.logical_or(mask_binary, gt_mask).sum()
            iou = intersection / union if union != 0 else 0
            
            
            total_iou += iou
            count += 1

            true_positive = intersection
            false_positive = mask_binary.sum() - true_positive
            false_negative = gt_mask.sum() - true_positive
            
            precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
            recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
            
            total_precision += precision
            total_recall += recall
            
            
            confidence = result.boxes[i].conf.item()  
            label = f"Human{confidence:.2f}"

            contours, _ = cv2.findContours(mask_np.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                cv2.drawContours(img_resized, [contour], -1, (0, 255, 0), 2)  
                
                M = cv2.moments(contour)
                if M['m00'] > 0:
                    cX = int(M['m10'] / M['m00'])
                    cY = int(M['m01'] / M['m00'])
                    cv2.putText(img_resized, label, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

final_output = cv2.addWeighted(img_resized, 0.7, output_mask, 0.3, 0)


cv2.imshow("Segmented Result", final_output)
cv2.waitKey(0)
cv2.destroyAllWindows()

if count > 0:
    mean_iou = total_iou / count
    mean_precision = total_precision / count
    mean_recall = total_recall / count
    f1_score = (2 * mean_precision * mean_recall) / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0
    
    print(f"Mean IoU: {mean_iou:.4f}")
    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
else:
    print("No valid masks found for metrics calculation.")



0: 640x640 1 human, 72.0ms
Speed: 5.7ms preprocess, 72.0ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)
Mean IoU: 0.0052
Mean Precision: 0.0190
Mean Recall: 0.0070
F1 Score: 0.0103


In [38]:
test_metrics = model.val(data="C:\\Users\\adhik\\Downloads\\camouflage\\work\\data.yaml", conf=0.25)
mean_results = test_metrics.mean_results()
print(f"Mean Precision: {mean_results[0]}, Mean Recall: {mean_results[1]}, Mean mAP: {mean_results[2]}")


Ultralytics 8.3.3  Python-3.10.14 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i7-12650H)


val: Scanning C:\Users\adhik\Downloads\camouflage\work\valid\labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.05it/s]


                   all         69        104      0.693      0.508      0.611      0.375      0.741      0.514      0.614      0.386
           camouflaged         17         20      0.613        0.6      0.606      0.351      0.619        0.6      0.606      0.368
                 human         56         84      0.774      0.417      0.616        0.4      0.862      0.429      0.621      0.404
Speed: 1.1ms preprocess, 57.0ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to C:\Users\adhik\runs\segment\val2
Mean Precision: 0.6932275186315373, Mean Recall: 0.5083333333333333, Mean mAP: 0.6109461858940368
